In [7]:
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

import scikitplot as skplt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [8]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 32

# Training 수
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 224
img_height = 224

ERROR! Session/line number was not unique in database. History logging moved to new session 610


## 디렉토리 경로 설정 필요!!

In [9]:
# 디렉토리 경로 설정 필요
CurrentDirectory = "./Data/CNN/"

train_directory = CurrentDirectory + 'TRAIN/'
test_directory  = CurrentDirectory + 'TEST/'
model_directory = CurrentDirectory + 'MODEL/'
tensorboard_directory = CurrentDirectory + 'Tensorboard'

In [10]:
# 모델 Return
VGGModel = tf.keras.applications.EfficientNetB0(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_width,img_height,3), pooling=None)
x = GlobalAveragePooling2D()(VGGModel.output)
predictions = Dense(3, activation='softmax')(x)

DeepLearning = Model(inputs=VGGModel.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
DeepLearning.compile(optimizer=
         SGD(lr=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc']
) # 나이를, MSE


In [11]:
DeepLearning.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

In [12]:
# Online-augmentation 적용 Generator
# 1. 이미지를 전부다 불러서 램 (메모리)에 올릴 수 없기 때문
# 2. 이미지는 Augmentation을 해주는게 좋아서

DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Generator의 instance 생성 (Train)
TRAIN_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "training")

VALID_GENERATOR = DATAGEN_TRAIN.flow_from_directory(
    train_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode= "categorical",
    subset = "validation")

Found 719 images belonging to 3 classes.
Found 79 images belonging to 3 classes.


In [ ]:
DeepLearning.fit(
    TRAIN_GENERATOR,
    epochs = 10,
    shuffle = True,
    validation_data = VALID_GENERATOR)

Epoch 1/10
10/23 [============>.................] - ETA: 2:36 - loss: 1.1281 - acc: 0.3656

**Test 데이터 읽기**

In [ ]:
# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

# Generator의 instance 생성 (Test)
TEST_GENERATOR = DATAGEN_TEST.flow_from_directory(
    test_directory,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
    class_mode= "categorical")

In [ ]:
TEST_GENERATOR.__getitem__(0)[1]

**모델 적합하기**

In [ ]:
# Call-back 함수

# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath=model_directory+'VGG19-{epoch:03d}-{val_loss:.4f}-{val_acc:.4f}.hdf5',
            monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# 학습과정 진행사항 확인
TB = TensorBoard(log_dir=tensorboard_directory, write_graph=True, write_images=True)

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='val_loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, TB, LR]

In [ ]:
########## Training Start
DeepLearning.fit(
        TRAIN_GENERATOR,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        
        epochs=15,
        callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=VALID_GENERATOR)
###########

 3/16 [====>.........................] - ETA: 51s - loss: 0.6923 - acc: 0.5417

In [ ]:
# average(Batch's Loss)
# Average(Loss of All data)

In [ ]:
DeepLearning.evaluate(VALID_GENERATOR)

In [ ]:
DeepLearning.evaluate(VALID_GENERATOR)

In [ ]:
# 학습하지 않았으므로, 강사가 학습한 모델 사용
DeepLearning.load_weights(model_directory+'PretrainedVGG.hdf5')

In [ ]:
DeepLearning.summary()

In [ ]:
# Prediction
TEST_Prediction = DeepLearning.predict_generator(TEST_GENERATOR, verbose=1)

In [ ]:
# data frame으로 변환
Result = pd.DataFrame(TEST_Prediction)

In [ ]:
skplt.metrics.plot_roc(TEST_GENERATOR.classes.tolist(), Result)

In [ ]:
# 다음과 같이 Back-bone model만 수정할 수 있다.
'''

Model = tf.keras.applications.VGG16(include_top=False,
    weights='imagenet', input_tensor=None, input_shape=(img_width,img_height,3), pooling=None)

Model = tf.keras.applications.ResNet101V2(include_top=False,
    weights='imagenet', input_tensor=None, input_shape=(img_width,img_height,3), pooling=None)

Model = tf.keras.applications.EfficientNetB4(include_top=False,
    weights='imagenet', input_tensor=None, input_shape=(img_width,img_height,3), pooling=None)
    
'''

https://www.tensorflow.org/api_docs/python/tf/keras/applications?hl=ko

# GradCam visualization 시각화 Heatmap

In [ ]:
import PIL

def grad_cam(model, img, layer_name="block5_conv3", label_name=None, category_id=None):
    img_tensor = np.expand_dims(img, axis=0)
    conv_layer = model.get_layer(layer_name)
    heatmap_model = tf.keras.Model(inputs=[model.inputs], outputs=[conv_layer.output, model.output])

    with tf.GradientTape() as gtape:
        conv_output, predictions = heatmap_model(img_tensor)
        if category_id == None:
            category_id = np.argmax(predictions[0])
        if label_name:
            print(label_name[category_id])
        output = predictions[:, category_id]
        grads = gtape.gradient(output, conv_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat
    return np.squeeze(heatmap), predictions

def grad_camAll(model, img, layer_name="conv4_block6_out", blockSize=7, label_name=None, category_id=None):
    img_tensor = np.expand_dims(img, axis=0)
    conv_layer = model.get_layer(layer_name)
    heatmap_model = Model([model.inputs], [conv_layer.output, model.output])

    with tf.GradientTape() as gtape:
        conv_output, predictions = heatmap_model(img_tensor)
        if category_id == None:
            category_id = np.argmax(predictions[0])
        if label_name:
            print(label_name[category_id])
        output = predictions[:, category_id]
        grads = gtape.gradient(output, conv_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    res = sp.ndimage.zoom(conv_output[0], (224/blockSize, 224/blockSize, 1), order=2)   
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, res), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat
    return np.squeeze(heatmap)

def show_imgwithheat(img_path, heatmap, alpha=0.4, return_array=False):
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = (heatmap*255).astype("uint8")
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    imgwithheat = PIL.Image.fromarray(superimposed_img)  
    display(imgwithheat)

    if return_array:
        return superimposed_img

In [ ]:
# CAM Generator
CAM_GENERATOR = DATAGEN_TEST.flow_from_directory(
    'as',
    target_size = (img_width, img_height),
    batch_size = 1,
    shuffle = False,
    class_mode='binary')

In [ ]:
DeepLearning.summary()

In [ ]:
for Iterator in range(0, 20):
    Image, Label = CAM_GENERATOR.next()
    Image = Image[0]
    FileName = CAM_GENERATOR.filenames[Iterator]
    # Final Layer
    HEATMAP1, pred1 = grad_cam(model = DeepLearning, img = Image, layer_name = 'block5_conv3', category_id=0)
    
    heatmap1 = cv2.resize(HEATMAP1, (Image.shape[1], Image.shape[0]))
    heatmap1 = (heatmap1*255).astype("uint8")
    heatmap1 = cv2.applyColorMap(heatmap1, cv2.COLORMAP_JET)

    fig, ax = plt.subplots( nrows=3, ncols=1 )  # create figure & 1 axis
    fig.set_size_inches(9.0, 3.0)
    ax = plt.subplot(1,3,1)
    ax.imshow(Image)
    ax = plt.subplot(1,3,2)
    ax.imshow(Image)
    ax.imshow(255-heatmap1, cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )
    ax = plt.subplot(1,3,3)
    ax.imshow(Image)
    ax.imshow(heatmap1, cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )

    plt.subplots_adjust(left = 0.06, wspace = 0.25, hspace = 0.1, bottom = 0.025, top = 0.975)
    pred=round(pred1.numpy()[0][0],3)
    HeatImage = "HEATMAP/0420_"+CAM_GENERATOR.filenames[Iterator].split('\\')[1] + "^" + str(round(pred, 4)) + ".jpg"
    plt.savefig(HeatImage.strip(), bbox_inches = 'tight')
    plt.close()
    plt.cla()
    plt.clf()

In [ ]:
MODEL = tf.keras.applications.VGG16(include_top=True, weights='imagenet')

In [ ]:
from tensorflow.keras.applications.vgg16 import decode_predictions
import requests

url = 'https://dimg.donga.com/wps/NEWS/IMAGE/2022/01/28/111500268.2.jpg'
image = PIL.Image.open(requests.get(url, stream=True).raw)
image = tf.keras.preprocessing.image.img_to_array(image)
image = cv2.resize(image, (224, 224) )
image = np.expand_dims(image, axis=0)
plt.imshow(image[0]/255, interpolation='nearest')
plt.show()
image = tf.keras.applications.vgg16.preprocess_input(image)

In [ ]:
res = MODEL.predict(image)

In [ ]:
category_id = 0
for task in [res[0] > 0.9][0]:
    if task == True:
        break
    category_id += 1

In [ ]:
Iterator=0

Image, Label = DATA_GENERATOR.__getitem__(0)
Image = Image[0]
FileName = DATA_GENERATOR.filenames[Iterator]

LayerNames = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
Heatmaps = []
for LayerIDX in range(0, len(LayerNames)):
    HEATMAP1, pred1 = grad_cam(model = MODEL, img = image[0], layer_name = LayerNames[LayerIDX], category_id=category_id)
    heatmap_resize = cv2.resize(HEATMAP1, (224, 224))
    heatmap_resize_255 = (heatmap_resize*255).astype("uint8")
    heatmap1 = cv2.applyColorMap(255-heatmap_resize_255, cv2.COLORMAP_JET ) / 255
    heatmap1[:, :, 2] -= .5
    heatmap1[(heatmap1[:, :, 2] < 0), 2] = 0
    Heatmaps.append(heatmap1)

fig, ax = plt.subplots( nrows=3, ncols=2 )  # create figure & 1 axis
fig.set_size_inches(18.0, 6.0)
ax = plt.subplot(2,3,1)
ax.imshow(Image)
ax = plt.subplot(2,3,2)
ax.imshow(Image)
ax.imshow(Heatmaps[0], cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )
ax = plt.subplot(2,3,3)
ax.imshow(Image)
ax.imshow(Heatmaps[1], cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )

ax = plt.subplot(2,3,4)
ax.imshow(Image)
ax.imshow(Heatmaps[2], cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )
ax = plt.subplot(2,3,5)
ax.imshow(Image)
ax.imshow(Heatmaps[3], cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )
ax = plt.subplot(2,3,6)
ax.imshow(Image)
ax.imshow(Heatmaps[4], cmap=plt.cm.jet, alpha=0.3, interpolation='nearest' )

plt.subplots_adjust(left = 0.06, wspace = -0.58, hspace = 0.3, bottom = 0.025, top = 0.975)
plt.show()